In [ ]:
#python libs
import math
import json
from dataclasses import dataclass, asdict
from typing import List
from typing import Any

#third party libs
import numpy as np
import cv2
from PIL import Image, ImageDraw
from rtree import index

from configs.redet.util import MatchExport
from configs.redet.util import Vehicle
from configs.redet.util import euclidean_distance
from configs.redet.util import Pnt, VehicleExport, crop_polygon, get_mean_color, rbbox_to_poly, upscale_2x

In [79]:
class Args:
    def __init__(self, dist_thr: float, width_thr: float, height_thr: float, color_thr: float):
        self.imgA = "./PA-SM-2020-06-12/satellite-sm.png"
        self.imgB = "./PA-SM-2020-10-17/satellite-sm.png"
        self.resA = "./pa-sm-2020-06-12-sm-gpu.npy"
        self.resB = "./pa-sm-2020-10-17-sm-gpu.npy"
        self.dist_thr = dist_thr
        self.width_thr = width_thr
        self.height_thr = height_thr
        self.color_thr = color_thr

In [80]:
args = Args(
    dist_thr=7.0,
    width_thr=6.0,
    height_thr=6.0,
    color_thr=10.0 )


In [81]:
def img_reg( a: Image, b: Image ):
    img_a = cv2.cvtColor( np.array( a ), cv2.COLOR_RGB2BGR )
    img_b = cv2.cvtColor( np.array( b ), cv2.COLOR_RGB2BGR )

    # convert the images to grayscale
    a_gray = cv2.cvtColor(img_a, cv2.COLOR_BGR2GRAY)
    b_gray = cv2.cvtColor(img_b, cv2.COLOR_BGR2GRAY)

    # create SIFT object to detect keypoints and compute descriptors
    sift = cv2.SIFT_create()
    kps1, descriptors1 = sift.detectAndCompute(a_gray, None)
    kps2, descriptors2 = sift.detectAndCompute(b_gray, None)

    # create and use flann matcher
    matcher = cv2.FlannBasedMatcher()
    matches = matcher.knnMatch(descriptors1, descriptors2, k=2)

    # apply ratio test to filter good matches
    good_matches = []
    ratio_threshold = 0.7
    for m, n in matches:
        if m.distance < ratio_threshold * n.distance:
            good_matches.append(m)

    # Extract the matched keypoints
    ref_points    = np.float32([kps1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    sensed_points = np.float32([kps2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    # Estimate the homography matrix using RANSAC
    homography, _ = cv2.findHomography(sensed_points, ref_points, cv2.RANSAC, 5.0)
    return homography

def translate( homography, coords ):
    coords_homogeneous = np.array([[coord[0], coord[1], 1] for coord in coords])
    translated_coords_homogeneous = homography.dot(coords_homogeneous.T).T
    translated_coords = translated_coords_homogeneous[:, :2] / translated_coords_homogeneous[:, 2, np.newaxis]
    return translated_coords

In [82]:
def compare_dimensions( a: Vehicle, b: Vehicle ):
    w_diff = abs( a.width - b.width   )
    h_diff = abs( a.height - b.height )
    # if w_diff > args.width_thr:
    #     return False
    # if h_diff > args.height_thr:
    #     return False
    return True

In [83]:
#img_b must be registered already
import PIL
PIL.Image.MAX_IMAGE_PIXELS = 9331200000

try:
    img_a 
except NameError:
    img_a = Image.open( args.imgA ).convert("RGBA")

try:
    img_b 
except NameError:
    img_b = Image.open( args.imgB ).convert("RGBA")
    
ident = identity_homography = np.array([[1, 0, 0],
                                        [0, 1, 0],
                                        [0, 0, 1]], dtype=np.float32)
try:
    homography 
except NameError:
    homography = ident #img_reg( args )

# cv2.imwrite("out/registered_image.png", img_registered )
# img_b = Image.open( "out/registered_image.png" ).convert("RGBA")

In [84]:
import torch
from mmrotate.core.bbox.transforms import norm_angle, obb2poly, obb2poly_le90, obb2poly_np_le90

def poly2obb_le90(polys):
    """Convert polygons to oriented bounding boxes.
    Args:
        polys (torch.Tensor): [x0,y0,x1,y1,x2,y2,x3,y3]
    Returns:
        obbs (torch.Tensor): [x_ctr,y_ctr,w,h,angle]
    """
    polys = torch.reshape(polys, [-1, 8])
    pt1, pt2, pt3, pt4 = polys[..., :8].chunk(4, 1)
    edge1 = torch.sqrt(
        torch.pow(pt1[..., 0] - pt2[..., 0], 2) +
        torch.pow(pt1[..., 1] - pt2[..., 1], 2))
    edge2 = torch.sqrt(
        torch.pow(pt2[..., 0] - pt3[..., 0], 2) +
        torch.pow(pt2[..., 1] - pt3[..., 1], 2))
    angles1 = torch.atan2((pt2[..., 1] - pt1[..., 1]),
                          (pt2[..., 0] - pt1[..., 0]))
    angles2 = torch.atan2((pt4[..., 1] - pt1[..., 1]),
                          (pt4[..., 0] - pt1[..., 0]))
    angles = polys.new_zeros(polys.shape[0])
    angles[edge1 > edge2] = angles1[edge1 > edge2]
    angles[edge1 <= edge2] = angles2[edge1 <= edge2]
    angles = norm_angle(angles, 'le90')
    x_ctr = (pt1[..., 0] + pt3[..., 0]) / 2.0
    y_ctr = (pt1[..., 1] + pt3[..., 1]) / 2.0
    edges = torch.stack([edge1, edge2], dim=1)
    width, _ = torch.max(edges, 1)
    height, _ = torch.min(edges, 1)
    return torch.stack([x_ctr, y_ctr, width, height, angles], 1)

def record_translate( rec: np.ndarray ):
    poly       = torch.Tensor( rbbox_to_poly( rec ) )
    translated = translate( homography, poly )
    rbb = poly2obb_le90( torch.Tensor( translated ) )
    return np.array( rbb[0], dtype=np.float64 )

A = np.load( args.resA )
B = np.load( args.resB )

#untransformed versions
a_entries = [Vehicle( r[0], r[1], r[2], r[3], r[4], r ) for r in A]
b_trans   = [record_translate( r ) for r in B]
b_entries = [Vehicle( r[0], r[1], r[2], r[3], r[4], r ) for r in b_trans]

#build spatial indices
a_qtree = index.Index()
for i, a in enumerate( a_entries ):
    a_qtree.insert( i, ( a.x, a.y ) )
b_qtree = index.Index()
for i, b in enumerate( b_entries ):
    b_qtree.insert( i, ( b.x, b.y ) )

In [85]:
matches: List[Vehicle] = []
for i, a in enumerate( a_entries ):
    b_ind = list( b_qtree.nearest( ( a.x, a.y ), 1, objects=True ) )[0].id
    b     = b_entries[b_ind]
    dist  = euclidean_distance( a.x, a.y, b.x, b.y )
    if dist > args.dist_thr:
        continue
    # if not compare_dimensions( a, b ):
    #     continue
    matches.append( [a, b] )

In [86]:
from PIL import Image, ImageDraw
import numpy as np

def crop_polygon_with_alpha(img: Image.Image, poly: np.ndarray) -> Image.Image:
    min_x, min_y = poly.min(axis=0).astype(int)
    max_x, max_y = poly.max(axis=0).astype(int)    
    crop = img.crop((min_x, min_y, max_x, max_y)).convert("RGBA")    
    local_poly = [(x - min_x, y - min_y) for x, y in poly]    
    mask = Image.new("L", crop.size, 0)
    ImageDraw.Draw(mask).polygon(local_poly, fill=255)    
    crop.putalpha(mask)
    return crop

In [87]:
import numpy as np
from typing import Dict, Tuple

# ❶ --- a minimal named-colour table -------------------------
CSS_NAMED_COLOURS: Dict[str, Tuple[int, int, int]] = {
    "black":       (0,   0,   0),
    "white":       (255, 255, 255),
    "red":         (255, 0,   0),
    "lime":        (0,   255, 0),
    "blue":        (0,   0,   255),
    "yellow":      (255, 255, 0),
    "cyan":        (0,   255, 255),
    "magenta":     (255, 0,   255),
    "gray":        (128, 128, 128),
    "orange":      (255, 165, 0),
    "brown":       (165, 42,  42),
    "pink":        (255, 192, 203),
    # …add as many as you like
}

# ❷ --- find nearest named colour ----------------------------
def nearest_named_colour(rgb: np.ndarray,
                         palette: Dict[str, Tuple[int, int, int]] = CSS_NAMED_COLOURS
                        ) -> str:
    """
    rgb      : (3,) uint8 array, e.g. array([123, 200,  34])
    palette  : mapping name -> (R, G, B)

    Returns the name with the smallest Euclidean distance in RGB space.
    """
    diffs = {name: np.linalg.norm(rgb - np.array(col)) for name, col in palette.items()}
    return min(diffs, key=diffs.get)

# ❸ --- glue code for your K-means output --------------------
def dominant_cluster_colour_name(centers: np.ndarray, counts: np.ndarray) -> str:
    """
    centers : (k,3) uint8   - cluster centroids from K-means
    counts  : (k,) int      - pixel counts per cluster

    Returns a named colour string for the most-populous cluster.
    """
    dominant_idx = np.argmax(counts)      # largest count
    dominant_rgb = centers[dominant_idx]  # uint8 triplet
    return nearest_named_colour(dominant_rgb)

In [88]:
from pathlib import Path
from typing import Tuple, Union
from sklearn.cluster import KMeans

def kmeans_palette(
    img: Image.Image,
    k: int = 8,
    max_iter: int = 100,
    random_state: int = 42,
) -> Tuple[str, Image.Image]:
    # ---- 1 · gather pixels --------------------------------------------------
    rgb = np.array(img.convert("RGB")).reshape(-1, 3).astype(np.float32)

    # ---- 2 · K-means --------------------------------------------------------
    km = KMeans(n_clusters=k, n_init="auto",
                max_iter=max_iter, random_state=random_state)
    labels = km.fit_predict(rgb)
    centers = km.cluster_centers_.astype(np.uint8)

    # pixel share per cluster (sorted largest->smallest)
    counts = np.bincount(labels)
    order = np.argsort(counts)[::-1]          # dominant first
    counts = counts[order]
    centers = centers[order]
    name = dominant_cluster_colour_name(centers, counts)

    # normalised bar widths
    frac = counts / counts.sum()
    size = (256, 32)
    w_px, h_px = size
    widths = np.round(frac * w_px).astype(int)

    # ---- 3 · paint bar chart ------------------------------------------------
    bar = Image.new("RGB", size)
    x = 0
    for w, colour in zip(widths, centers):
        if w == 0:            # very tiny clusters
            continue
        block = Image.new("RGB", (w, h_px), tuple(colour))
        bar.paste(block, (x, 0))
        x += w
    # if rounding left a gap at the end, fill it with the most dominant colour
    if x < w_px:
        bar.paste(Image.new("RGB", (w_px - x, h_px), tuple(centers[0])), (x, 0))

    return (name, bar)

In [90]:
lst: List[MatchExport] = []
i: int = 0

for match in matches:
    a: Vehicle = match[0]
    b: Vehicle = match[1]
    crop_a = rbbox_to_poly( a.arr )
    crop_b = rbbox_to_poly( b.arr )
    va = VehicleExport( Pnt( a.x, a.y ), a.width, a.height, a.theta, crop_a.tolist(), str(i) )
    vb = VehicleExport( Pnt( b.x, b.y ), b.width, b.height, b.theta, crop_b.tolist(), str(i) )

    #get crop
    img_crop_a = crop_polygon_with_alpha( img_a, crop_a )
    img_crop_b = crop_polygon_with_alpha( img_b, crop_b )
    # img_crop_a = img_crop_a.resize( upscale_2x( img_crop_a ), resample=Image.BOX )
    # img_crop_b = img_crop_b.resize( upscale_2x( img_crop_b ), resample=Image.BOX )
    (col_a_name, palette_a) = kmeans_palette(img_crop_a, k=4 )   # -> MyImage_a.kmeans.png
    (col_b_name, palette_b) = kmeans_palette(img_crop_b, k=4 )   # -> MyImage_b.kmeans.png

    #comparing color
    col_a = np.array( get_mean_color( img_crop_a.convert( "RGB" ) ) )
    col_b = np.array( get_mean_color( img_crop_b.convert( "RGB" ) ) )
    color_dist = abs( np.linalg.norm( col_a - col_b ) )

    #color distance
    # if color_dist > args.color_thr:
    #    continue

    img_crop_a.save( f"out/{i}a-{col_a_name}.png" )
    palette_a.save( f"out/{i}a.kmeans.png" )
    img_crop_b.save( f"out/{i}b-{col_b_name}.png" )
    palette_b.save( f"out/{i}b.kmeans.png" )
    dist = euclidean_distance( va.center.x, va.center.y, vb.center.x, vb.center.y)
    height_diff = abs( va.height - vb.height )
    width_diff  = abs( va.width  - vb.width )
    theta_diff  = abs( va.theta  - vb.theta )
    lst.append( MatchExport( va, vb, dist, color_dist, height_diff, width_diff, theta_diff) )
    # print( f"{dist} = dist( {col_a}, {col_b} )" )
    i += 1

vehicle_matches = json.dumps( list( map( lambda el: asdict( el ), lst ) ) )
with open( "matches-sm-3.json", "w" ) as f:
    f.write( vehicle_matches )

print( f"There were {len(matches)}" )

There were 403
